In [4]:
import json
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score, f1_score
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from sklearn.metrics import ndcg_score, average_precision_score

# Cargar el corpus
with open("01tdmatrix_corpus.json", "r") as f:
    data = json.load(f)

# Asumiendo que data es una lista de documentos
documentos = {doc["id"]: doc["text"] for doc in data}

# Definir consultas y juicios de relevancia
consultas = [
    "inteligencia artificial en medicina",
    "beneficios de la educación a distancia",
    "realidad aumentada en videojuegos",
    "desarrollo personal y hábitos saludables",
    "futuro del comercio electrónico",
    "tecnologías en cine moderno",
    "competencias de e-sports",
    "diagnóstico con dispositivos portátiles",
    "literatura de ciencia ficción",
    "plataformas de streaming"
]

juicios_relevancia = {
    "inteligencia artificial en medicina": [1, 4, 5],
    "beneficios de la educación a distancia": [2, 6, 9],
    "realidad aumentada en videojuegos": [3, 7],
    "desarrollo personal y hábitos saludables": [8, 10],
    "futuro del comercio electrónico": [11, 13, 15],
    "tecnologías en cine moderno": [14, 16],
    "competencias de e-sports": [17, 19],
    "diagnóstico con dispositivos portátiles": [20, 21],
    "literatura de ciencia ficción": [22, 23, 24],
    "plataformas de streaming": [25, 26]
}

# Sistema 1: Recuperación con TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(documentos.values())

# Sistema 2: Recuperación con Word2Vec
processed_docs = [simple_preprocess(doc) for doc in documentos.values()]
w2v_model = Word2Vec(sentences=processed_docs, vector_size=100, window=5, min_count=1, workers=4)
word_vectors = w2v_model.wv

# Función para obtener vector promedio de Word2Vec
def get_w2v_vector(text):
    words = simple_preprocess(text)
    vector = np.mean([word_vectors[w] for w in words if w in word_vectors], axis=0)
    return vector

# Calcular métricas de evaluación
def calcular_métricas(resultados, relevancia):
    k = len(relevancia)
    relevantes_en_top_k = sum([1 for i in resultados[:k] if i in relevancia])
    precision_at_k = relevantes_en_top_k / k
    recall = relevantes_en_top_k / len(relevancia)
    f1 = 2 * (precision_at_k * recall) / (precision_at_k + recall) if (precision_at_k + recall) > 0 else 0
    return precision_at_k, recall, f1

# Evaluación y comparación
resultados_sistema_1 = {}
resultados_sistema_2 = {}
metricas_sistema_1 = {}
metricas_sistema_2 = {}

for consulta in consultas:
    # Sistema 1: Similaridad con TF-IDF
    consulta_vec = vectorizer.transform([consulta])
    similitudes_tfidf = cosine_similarity(consulta_vec, tfidf_matrix).flatten()
    resultados_tfidf = [doc_id for doc_id, _ in sorted(zip(documentos.keys(), similitudes_tfidf), key=lambda x: -x[1])]
    resultados_sistema_1[consulta] = resultados_tfidf

    # Sistema 2: Similaridad con Word2Vec
    consulta_vec_w2v = get_w2v_vector(consulta)
    similitudes_w2v = [
        cosine_similarity(consulta_vec_w2v.reshape(1, -1), get_w2v_vector(doc).reshape(1, -1))[0][0]
        for doc in documentos.values()
    ]
    resultados_w2v = [doc_id for doc_id, _ in sorted(zip(documentos.keys(), similitudes_w2v), key=lambda x: -x[1])]
    resultados_sistema_2[consulta] = resultados_w2v

    # Calcular métricas para cada sistema
    relevancia = juicios_relevancia[consulta]

    prec_k_tfidf, recall_tfidf, f1_tfidf = calcular_métricas(resultados_tfidf, relevancia)
    prec_k_w2v, recall_w2v, f1_w2v = calcular_métricas(resultados_w2v, relevancia)

    map_tfidf = average_precision_score([1 if doc in relevancia else 0 for doc in resultados_tfidf], similitudes_tfidf)
    map_w2v = average_precision_score([1 if doc in relevancia else 0 for doc in resultados_w2v], similitudes_w2v)

    ndcg_tfidf = ndcg_score([relevancia], [resultados_tfidf[:len(relevancia)]])
    ndcg_w2v = ndcg_score([relevancia], [resultados_w2v[:len(relevancia)]])

    metricas_sistema_1[consulta] = {
        "Prec@k": prec_k_tfidf, "Recall": recall_tfidf, "F1-score": f1_tfidf, "MAP": map_tfidf, "nDCG": ndcg_tfidf
    }
    metricas_sistema_2[consulta] = {
        "Prec@k": prec_k_w2v, "Recall": recall_w2v, "F1-score": f1_w2v, "MAP": map_w2v, "nDCG": ndcg_w2v
    }

# Comparación de resultados
for consulta in consultas:
    print(f"Consulta: {consulta}")
    print("Sistema 1 (TF-IDF):", metricas_sistema_1[consulta])
    print("Sistema 2 (Word2Vec):", metricas_sistema_2[consulta])
    print()


Consulta: inteligencia artificial en medicina
Sistema 1 (TF-IDF): {'Prec@k': 0.0, 'Recall': 0.0, 'F1-score': 0, 'MAP': 0.5694444444444444, 'nDCG': 0.7670568668842022}
Sistema 2 (Word2Vec): {'Prec@k': 0.3333333333333333, 'Recall': 0.3333333333333333, 'F1-score': 0.3333333333333333, 'MAP': 0.8095238095238095, 'nDCG': 0.888731240572507}

Consulta: beneficios de la educación a distancia
Sistema 1 (TF-IDF): {'Prec@k': 0.3333333333333333, 'Recall': 0.3333333333333333, 'F1-score': 0.3333333333333333, 'MAP': 0.225, 'nDCG': 0.9999999999999999}
Sistema 2 (Word2Vec): {'Prec@k': 0.3333333333333333, 'Recall': 0.3333333333333333, 'F1-score': 0.3333333333333333, 'MAP': 1.0, 'nDCG': 0.9620096455530144}

Consulta: realidad aumentada en videojuegos
Sistema 1 (TF-IDF): {'Prec@k': 0.5, 'Recall': 0.5, 'F1-score': 0.5, 'MAP': 1.0, 'nDCG': 0.9999999999999998}
Sistema 2 (Word2Vec): {'Prec@k': 0.0, 'Recall': 0.0, 'F1-score': 0, 'MAP': 0.3333333333333333, 'nDCG': 0.9999999999999998}

Consulta: desarrollo person

C:\Users\kale\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
C:\Users\kale\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
C:\Users\kale\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
C:\Users\kale\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
C:\Users\kale\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_ranking.py:1030: UserWarning: No positive class found in y_true